## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [6]:
len(documents)

31

In [7]:
documents[24]

Document(metadata={'source': 'knowledge-base\\employees\\Oliver Spencer.md', 'doc_type': 'employees'}, page_content='# HR Record\n\n# Oliver Spencer\n\n## Summary\n- **Date of Birth**: May 14, 1990  \n- **Job Title**: Backend Software Engineer  \n- **Location**: Austin, Texas  \n\n## Insurellm Career Progression\n- **March 2018**: Joined Insurellm as a Backend Developer I, focusing on API development for customer management systems.\n- **July 2019**: Promoted to Backend Developer II after successfully leading a team project to revamp the claims processing system, reducing response time by 30%.\n- **June 2021**: Transitioned to Backend Software Engineer with a broader role in architecture and system design, collaborating closely with the DevOps team.\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January 2023**: Awarded a mentorship role to guide new hires in backend technology

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [10]:
len(chunks)

98

In [11]:
chunks[6]

Document(metadata={'source': 'knowledge-base\\contracts\\Contract with Apex Reinsurance for Rellm.md', 'doc_type': 'contracts'}, page_content='4. **Escalation Protocol**: Issues that cannot be resolved at the first level of support will be escalated to the senior support team, ensuring that critical concerns are addressed promptly.\n\n---\n\n**Acceptance of Terms**: By signing below, both parties agree to the Terms, Renewal, Features, and Support outlined in this Agreement.\n\n**Insurellm, Inc.**  \n_____________________________  \nAuthorized Signature   \nDate: ___________________  \n\n**Apex Reinsurance**  \n_____________________________  \nAuthorized Signature  \nDate: ___________________')

In [12]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, contracts, products, employees


In [ ]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")